# Diseño y Vlsm 

# 1. Diseño de Red y VLSM - Grupo 10
**Proyecto:** Infraestructura Tecnológica "Carnaval Conectado Brasil"

**Topología:** Malla Parcial (Partial Mesh)

**Red Base:** `10.2.0.0/16`

## 1.1 Justificación de la Topología Física
Se ha implementado una topología de **Malla Parcial** que equilibra redundancia y costo.

* **Conexión Central:** Cada sede remota tiene un enlace directo único hacia la Sede Central (Río de Janeiro) para el tráfico de servicios críticos.
* **Interconexión Lateral:** Se han desplegado enlaces directos entre **São Paulo ↔ Salvador** y **Salvador ↔ Recife**.
* **Beneficio:** Esta arquitectura permite que, si el enlace directo "Recife ↔ Río" falla, el tráfico pueda viajar por la ruta "Recife → Salvador → Río", garantizando alta disponibilidad sin duplicar todos los cables centrales.

## 1.2 Estrategia de Direccionamiento IP (Cálculos)

### A. Requisito Crítico: Capacidad Masiva (Río Admin)
El diseño exige que la subred administrativa principal soporte un mínimo de **10,000 hosts**.
$$2^n - 2 \geq 10,000$$
* Con $n=14 \rightarrow 16,384$ hosts (Suficiente).
* **Máscara Resultante:** $32 - 14 = \mathbf{/18}$ (`255.255.192.0`).

### B. Requisito de Alta Densidad (Wi-Fi)
Para las redes inalámbricas de invitados en cada sede, se estiman hasta 500 conexiones simultáneas.
$$2^n - 2 \geq 500$$
* Con $n=9 \rightarrow 510$ hosts.
* **Máscara Resultante:** $\mathbf{/23}$ (`255.255.254.0`).

### C. Segmentación VLSM
Se utiliza el bloque asignado `10.2.0.0/16`.
1.  **Red Masiva:** Se asigna el bloque `10.2.64.0/18` a la VLAN crítica de Río.
2.  **Redes Estándar:** Las VLANs departamentales usan máscaras `/24`.
3.  **Gestión Out-of-Band:** Se reserva una `/30` para el puerto de administración física.

## 1.3 Tabla de Direccionamiento LAN
Distribución de subredes para configuración de VLANs y DHCP.

| Sede | VLAN / Uso | Máscara | Dirección de Red | Primera IP (Gateway) | Última IP Útil | Broadcast |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **Río** | **10 Admin (Critica)** | **/18** | **`10.2.64.0`** | `10.2.64.1` | `10.2.127.254` | `10.2.127.255` |
| **Río** | 20 Servidores | /24 | `10.2.128.0` | `10.2.128.1` | `10.2.128.254` | `10.2.128.255` |
| **Río** | 30 Seguridad | /24 | `10.2.129.0` | `10.2.129.1` | `10.2.129.254` | `10.2.129.255` |
| **Río** | 50 IoT | /24 | `10.2.130.0` | `10.2.130.1` | `10.2.130.254` | `10.2.130.255` |
| **Río** | **Cable Admin** | **/30** | `10.2.131.0` | `10.2.131.1` | `10.2.131.2` | `10.2.131.3` |
| **SP** | 30 Prensa | /23 | `10.2.132.0` | `10.2.132.1` | `10.2.133.254` | `10.2.133.255` |
| **SP** | 40 Producción | /24 | `10.2.134.0` | `10.2.134.1` | `10.2.134.254` | `10.2.134.255` |
| **Sal** | 20 Logística | /24 | `10.2.135.0` | `10.2.135.1` | `10.2.135.254` | `10.2.135.255` |
| **Sal** | 50 IoT | /24 | `10.2.136.0` | `10.2.136.1` | `10.2.136.254` | `10.2.136.255` |
| **Rec** | 40 Seguridad | /24 | `10.2.137.0` | `10.2.137.1` | `10.2.137.254` | `10.2.137.255` |
| **Rec** | 99 Soporte | /24 | `10.2.138.0` | `10.2.138.1` | `10.2.138.254` | `10.2.138.255` |
| **Río** | **60 Wi-Fi Invitados** | **/23** | **`10.2.140.0`** | `10.2.140.1` | `10.2.141.254` | `10.2.141.255` |
| **SP** | **60 Wi-Fi Invitados** | **/23** | **`10.2.142.0`** | `10.2.142.1` | `10.2.143.254` | `10.2.143.255` |
| **Sal** | **60 Wi-Fi Invitados** | **/23** | **`10.2.144.0`** | `10.2.144.1` | `10.2.145.254` | `10.2.145.255` |
| **Rec** | **60 Wi-Fi Invitados** | **/23** | **`10.2.146.0`** | `10.2.146.1` | `10.2.147.254` | `10.2.147.255` |

## 1.4 Direccionamiento Enlaces WAN (Backbone)
Se utiliza el bloque reservado `10.2.200.0/24` con máscaras `/30`.

| Enlace | Origen (DCE) | Destino | Dirección de Red | IP Origen | IP Destino |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **Río ↔ SP** | Río `Se0/0/0` | SP `Se0/0/0` | **`10.2.200.0`** | `.1` | `.2` |
| **Río ↔ Salvador** | Río `Se0/1/0` | Sal `Se0/0/0` | **`10.2.200.4`** | `.5` | `.6` |
| **Río ↔ Recife** | Río `Se0/2/0` | Rec `Se0/0/0` | **`10.2.200.8`** | `.9` | `.10` |
| **SP ↔ Salvador** | SP `Se0/0/1` | Sal `Se0/0/1` | **`10.2.200.12`** | `.13` | `.14` |
| **Salvador ↔ Recife**| Sal `Se0/1/0` | Rec `Se0/0/1` | **`10.2.200.16`** | `.17` | `.18` |